In [6]:
import numpy as np
from scipy.stats import norm
from numpy import linalg as nl
import pandas as pd
from model.pcr import PCR

In [16]:
def qf(A, x):
    return np.sum(x * np.dot(A, x))

In [27]:
def msig(p, pho):
    if p == 1:
        return np.array(np.arange(1, 2, dtype=float)).reshape(1,1)
    else:
        mat_left = np.power(rho, np.arange(p - 1, 0, -1)).reshape(p - 1, 1)
        mat_below = np.power(rho, np.arange(p - 1, -1, -1)).reshape(1, p)
        mat_above = np.c_[msig(p - 1, rho), mat_left]
        return np.r_[mat_above, mat_below]

In [28]:
def sim(n, p, pho, mu, beta0, beta1):
    var = msig(p, pho)
    up_mat = nl.cholesky(var)
    x = norm .rvs(size = (n, p))
    x = np.dot(x, up_mat)
    x = x + mu
    y = beta0 + np.dot(x, beta1) + norm.rvs(size = n)
    return x, y

In [57]:
def pls_pure(xtx, xty, x_mean, y_mean, n, p, is_scale = True):
    xtx_scale = xtx - np.outer(x_mean, x_mean)*n
    xty_scale = xty - x_mean*y_mean*n
    x_std = None
    if is_scale:
        x_std = np.sqrt(np.diag(xtx_scale) / (n - 1))
        x_std_mat = 1 / np.repeat(x_std.reshape((1, p)), p, axis=0)
        xtx_scale = x_std_mat.T * xtx_scale * x_std_mat
        xty_scale = xty_scale / x_std
    ete = np.copy(xtx_scale)
    etf = np.copy(xty_scale)
    wp_mat = np.eye(p)
    b_ps = []
    for i in np.arange(p):
        etf_norm = nl.norm(etf)
        w = etf / etf_norm
        t_norm = qf(ete, w)
        r = etf_norm / t_norm
        ws = r * np.dot(wp_mat, w)
        b_ps.append(np.copy(ws)) # list
        pp = np.dot(ete, w) / t_norm
        if i < (p - 1):
            wp = np.eye(p) - np.outer(w, pp)
            wpt = wp.T
            ete = np.dot(wpt, np.dot(ete, wp))
            etf = np.dot(wpt, etf)
            wp_mat = np.dot(wp_mat, wp)
    b_ps = np.array(b_ps)
    b_ps = np.cumsum(b_ps, axis=0)
    if is_scale:
        b_ps = b_ps / x_std
    b0 = y_mean - np.dot(b_ps, x_mean)
    b_ps = np.c_[b0.reshape((p, 1)), b_ps]
    return b_ps

In [73]:
class PLS(object):
    
    def __init__(self, x, y, x_names, is_scale=True):
        self.x = x
        self.y = y
        self.n, self.p = np.shape(x)
        self.xtx = np.dot(x.T, x)
        self.xty = np.dot(x.T, y)
        self.yty = np.sum(y*y)
        self.b = np.array([0])
        self.is_scale = is_scale
        self.x_mean = np.mean(x, axis=0)
        self.y_mean = np.mean(y)
        self.cv_err = 0
        self.cv_b = 0
        self.x_names = x_names

    def pls(self):
        self.b = pls_pure(self.xtx, self.xty, self.x_mean, self.y_mean, self.n,
                                                          self.p, self.is_scale)

    def cv(self, k=10):
        indexs = np.array_split(np.random.permutation(np.arange(0, self.n)), k)

        def cvk(index):
            x_cv = self.x[index]
            n_cv, p_cv = np.shape(x_cv)
            if n_cv == 1:
                x_cv = x_cv.reshape((1, self.p))
            tn_ = self.n - n_cv
            y_cv = self.y[index]
            x_t = x_cv.T
            xtx_ = self.xtx - np.dot(x_t, x_cv)
            xty_ = self.xty - np.dot(x_t, y_cv)
            x_sum = np.sum(x_cv, axis = 0)
            y_sum = np.sum(y_cv)
            x_mean_ = (self.n * self.x_mean - x_sum) / tn_
            y_mean_ = (self.n * self.y_mean - y_sum) / tn_
            b_cv = pls_pure(xtx_, xty_, x_mean_, y_mean_, tn_, p_cv, self.is_scale)
            x_cv = np.c_[np.ones((n_cv, 1)), x_cv]
            ty_pred = np.dot(b_cv, x_cv.T)
            err = (ty_pred - y_cv)**2
            return np.sum(err, axis = 1)
        self.cv_err = np.sum(np.array([cvk(index) for index in indexs]), axis=0)/self.n
        min_k = np.argmin(self.cv_err)
        self.cv_b = self.b[min_k]
        return self.cv_b

    def report_coe(self):
        names = np.append("inter", self.x_names)
        results = pd.DataFrame(self.b, columns=names, index=np.arange(1, self.p + 1))
        results["cverr"] = self.cv_err
        return results

    def predict(self, xn):
        tn, _ = np.shape(xn)
        xn_ = np.c_[np.ones((tn, 1)), xn]
        return np.dot(self.cv_b, xn_.T)

    def predict_err(self, xn, yn):
        err = (yn - self.predict(xn))**2
        return np.mean(err)

    def test_err(self, xn, yn):
        tn, _ = np.shape(xn)
        xn_ = np.c_[np.ones((tn, 1)), xn]
        err = yn - np.dot(self.b, xn_.T)
        err = err * err
        err_mean = np.mean(err, axis=1)
        err_std = np.std(err, axis=1, ddof=1) / np.sqrt(tn)
        result = {'err_mean': err_mean, 'err_std': err_std}
        return pd.DataFrame(result)

一、改变变量个数P

In [75]:
n, rho = 100, 0.5
n1 = 2000

In [86]:
beta0 = 0.5
p_1, p_2, p_3 = 10, 30, 50

mu_1 = norm.rvs(size=p_1, scale=1)
beta1_1 = 0.5 * np.ones(p_1, dtype=float)
x_p1, y_p1 = sim(n, p_1, rho, mu_1, beta0, beta1_1)
x1_1, y1_1 = sim(n1, p_1, rho, mu_1, beta0, beta1_1)
names = list(range(p_1))

mu_2 = norm.rvs(size=p_2, scale=1)
beta1_2 = 0.5 * np.ones(p_2, dtype=float)
x_p2, y_p2 = sim(n, p_2, rho, mu_2, beta0, beta1_2)
x1_2, y1_2 = sim(n1, p_2, rho, mu_2, beta0, beta1_2)
names = list(range(p_2))

mu_3 = norm.rvs(size=p_3, scale=1)
beta1_3 = 0.5 * np.ones(p_3, dtype=float)
x_p3, y_p3 = sim(n, p_3, rho, mu_3, beta0, beta1_3)
x1_3, y1_3 = sim(n1, p_3, rho, mu_3, beta0, beta1_3)
names = list(range(p_3))

In [87]:
pcr1 = PCR(x_p1, y_p1, names, is_scale=True)
pcr1.pcr()
#pcr1.report_coe()
pcr1.test_err(x1_1, y1_1)

,err_mean,err_std
0,2.218381,0.070766
1,2.108459,0.066811
2,1.250596,0.037323
3,1.142904,0.034414
4,1.098395,0.033537
5,1.091321,0.033378
6,1.111585,0.033943
7,1.120931,0.034250
8,1.130138,0.034734
9,1.129375,0.034720


In [94]:
is_scale = False
is_var_exp = True
pls1 = PLS(x_p1, y_p1, names, is_scale)
pls1.pls()
#pls1.cv()
pls1.test_err(x1_1, y1_1)

,err_mean,err_std
0,1.294172,0.040095
1,1.121278,0.033950
2,1.109706,0.034160
3,1.129537,0.034698
4,1.130777,0.034731
5,1.129575,0.034720
6,1.129277,0.034718
7,1.129320,0.034719
8,1.129384,0.034720
9,1.129375,0.034720


In [95]:
pcr2 = PCR(x_p2, y_p2, names, is_scale=True, is_var_exp=True)
pcr2.pcr()
#pcr2.report_coe()
pcr2.test_err(x1_2, y1_2)

,err_mean,err_std
0,17.660248,0.572433
1,12.691005,0.404276
2,12.491381,0.398312
3,2.864966,0.088185
4,2.819942,0.088055
5,1.977570,0.061558
6,1.381793,0.041817
7,1.381707,0.041815
8,1.388843,0.043288
9,1.374547,0.043385


In [97]:
pls2 = PLS(x_p2, y_p2, names, is_scale)
pls2.pls()
#pls2.report_coe()
pls2.test_err(x1_2, y1_2)

,err_mean,err_std
0,2.581856,0.078832
1,1.453328,0.045261
2,1.326780,0.040893
3,1.297969,0.039787
4,1.290479,0.039589
5,1.291159,0.039457
6,1.303114,0.039797
7,1.299934,0.039638
8,1.302657,0.039719
9,1.301225,0.039653


In [98]:
pcr3 = PCR(x_p3, y_p3, names, is_scale=True, is_var_exp=True)
pcr3.pcr()
# pcr3.b
#pcr3.report_coe()
pcr3.test_err(x1_3, y1_3)

,err_mean,err_std
0,37.171448,1.185765
1,33.100075,1.064004
2,32.116777,1.042413
3,8.787325,0.273369
4,5.066995,0.158027
5,4.196425,0.136519
6,4.120400,0.134092
7,3.350743,0.109982
8,3.181257,0.100847
9,3.291992,0.104222


In [103]:
pls3 = PLS(x_p3, y_p3, names, is_scale)
pls3.pls()
pls3.predict_err(x1_3, y1_3)
pls3.test_err(x1_3, y1_3)

,err_mean,err_std
0,3.303174,0.108481
1,1.931859,0.064043
2,1.519625,0.049577
3,1.427083,0.045306
4,1.391203,0.044398
5,1.429864,0.046038
6,1.489253,0.048867
7,1.532073,0.050354
8,1.550657,0.051309
9,1.567581,0.052030


变量个数越多，测试误差越大。偏最小二乘回归的测试误差范围小于主成分回归，更为精确。

二、改变相关系数ρ

In [106]:
rho_1, rho_2, rho_3 = 0.25, 0.5, 0.75

mu = norm.rvs(size=p_2, scale=1)
beta0, beta1 = 0.5, 0.5 * np.ones(p_2, dtype=float)
names = list(range(p_2))

x_r1, y_r1 = sim(n, p_2, rho_1, mu, beta0, beta1)
x1_1, y1_1 = sim(n1, p_2, rho_1, mu, beta0, beta1)

x_r2, y_r2 = sim(n, p_2, rho_2, mu, beta0, beta1)
x1_2, y1_2 = sim(n1, p_2, rho_2, mu, beta0, beta1)

x_r3, y_r3 = sim(n, p_2, rho_3, mu, beta0, beta1)
x1_3, y1_3 = sim(n1, p_2, rho_3, mu, beta0, beta1)


In [107]:
pcr1 = PCR(x_r1, y_r1, names, is_scale=True, is_var_exp=True)
pcr1.pcr()
#pcr1.report_coe()
pcr1.test_err(x1_1, y1_1)

,err_mean,err_std
0,22.510462,0.679155
1,22.064755,0.672990
2,5.389839,0.173195
3,5.382364,0.172811
4,2.968747,0.093586
5,2.965279,0.093974
6,1.775296,0.059799
7,1.652359,0.056311
8,1.562325,0.052485
9,1.564739,0.052460


In [108]:
pls1 = PLS(x_r1, y_r1, names, is_scale)
pls1.pls()
#pls1.cv(n)
#pls1.report_coe()
pls1.test_err(x1_1, y1_1)

,err_mean,err_std
0,2.667055,0.083797
1,1.446843,0.048352
2,1.224981,0.040850
3,1.244031,0.041476
4,1.267912,0.042633
5,1.309559,0.043685
6,1.384166,0.045738
7,1.415926,0.046626
8,1.429799,0.046939
9,1.440858,0.047219


In [109]:
pcr2 = PCR(x_r2, y_r2, names, is_scale=True, is_var_exp=True)
pcr2.pcr()
#pcr2.report_coe()
pcr2.test_err(x1_2, y1_2)

,err_mean,err_std
0,9.527280,0.322381
1,2.285570,0.071356
2,2.059528,0.064577
3,2.062430,0.064679
4,1.946129,0.061206
5,1.816789,0.056274
6,1.611458,0.050145
7,1.611671,0.050102
8,1.556178,0.047936
9,1.438909,0.044604


In [111]:
pls2 = PLS(x_r2, y_r2, names, is_scale)
pls2.pls()
pls2.cv(n)
pls2.report_coe()
pls2.test_err(x1_2, y1_2)

,err_mean,err_std
0,1.999530,0.062001
1,1.507776,0.046225
2,1.373692,0.042743
3,1.407731,0.043625
4,1.511183,0.046143
5,1.596109,0.048467
6,1.616975,0.049206
7,1.686482,0.051536
8,1.684094,0.051526
9,1.691220,0.051815


In [112]:
pcr3 = PCR(x_r3, y_r3, names, is_scale=True, is_var_exp=True)
pcr3.pcr()
#pcr3.report_coe()
pcr3.test_err(x1_3, y1_3)

,err_mean,err_std
0,17.355410,0.552055
1,13.966702,0.450157
2,11.139945,0.347027
3,6.515963,0.203361
4,6.367674,0.197498
5,4.463416,0.140394
6,1.565469,0.050392
7,1.552260,0.050048
8,1.483964,0.048155
9,1.483997,0.048154


In [113]:
pls3 = PLS(x_r3, y_r3, names, is_scale)
pls3.pls()
#pls3.report_coe()
pls3.test_err(x1_3, y1_3)

,err_mean,err_std
0,3.059596,0.096729
1,1.226094,0.040532
2,1.135961,0.038504
3,1.074540,0.035722
4,1.099067,0.036861
5,1.121248,0.037750
6,1.116924,0.037647
7,1.117283,0.037683
8,1.117119,0.037706
9,1.120282,0.037826


三、改变β的取值

In [115]:
mu = norm.rvs(size=p_2, scale=1)
beta0_1, beta0_2, beta0_3 = 0.1, 0.5, 1
beta1_1, beta1_2, beta1_3 = 0.1 * np.ones(p_2, dtype=float), 0.5 * np.ones(p_2, dtype=float), 0.1 * np.ones(p_2, dtype=float)
names = list(range(p_2))

x_b1, y_b1 = sim(n, p_2, rho_3, mu, beta0_1, beta1_1)
x1_1, y1_1 = sim(n1, p_2, rho_3, mu, beta0_1, beta1_1)

x_b2, y_b2 = sim(n, p_2, rho_3, mu, beta0_2, beta1_2)
x1_2, y1_2 = sim(n1, p_2, rho_3, mu, beta0_2, beta1_2)

x_b3, y_b3 = sim(n, p_2, rho_3, mu, beta0_3, beta1_3)
x1_3, y1_3 = sim(n1, p_2, rho_3, mu, beta0_3, beta1_3)

In [116]:
pcr1 = PCR(x_b1, y_b1, names, is_scale=True, is_var_exp=True)
pcr1.pcr()
#pcr1.report_coe()
pcr1.test_err(x1_1, y1_1)

,err_mean,err_std
0,1.684313,0.052950
1,1.065187,0.033989
2,1.079481,0.034162
3,1.091773,0.034599
4,1.093944,0.034636
5,1.101482,0.034894
6,1.104240,0.034987
7,1.103806,0.035397
8,1.104068,0.035408
9,1.048086,0.033956


In [117]:
pls1 = PLS(x_b1, y_b1, names, is_scale)
pls1.pls()
pls1.report_coe()
pls1.test_err(x1_1, y1_1)

,err_mean,err_std
0,1.029109,0.033279
1,1.056203,0.034047
2,1.151131,0.037642
3,1.202747,0.039063
4,1.294778,0.041615
5,1.348112,0.043419
6,1.368402,0.044009
7,1.375297,0.044113
8,1.379267,0.044276
9,1.383116,0.044451


In [118]:
pcr2 = PCR(x_b2, y_b2, names, is_scale=True, is_var_exp=True)
pcr2.pcr()
#pcr2.report_coe()
pcr2.test_err(x1_2, y1_2)

,err_mean,err_std
0,6.789957,0.213055
1,4.481985,0.142341
2,4.618290,0.145121
3,4.011242,0.123496
4,3.627850,0.115424
5,2.047173,0.065602
6,1.889637,0.060394
7,1.884113,0.060192
8,1.728108,0.055630
9,1.737774,0.055999


In [119]:
pls2 = PLS(x_b2, y_b2, names, is_scale)
pls2.pls()
#pls2.report_coe()
pls2.test_err(x1_2, y1_2)

,err_mean,err_std
0,2.679696,0.084348
1,1.515704,0.048509
2,1.300073,0.041495
3,1.351155,0.043653
4,1.407996,0.045074
5,1.441071,0.045839
6,1.453210,0.046110
7,1.476817,0.046621
8,1.504273,0.047429
9,1.504173,0.047477


In [120]:
pcr3 = PCR(x_b3, y_b3, names, is_scale=True, is_var_exp=True)
pcr3.pcr()
#pcr3.report_coe()
pcr3.test_err(x1_3, y1_3)

,err_mean,err_std
0,1.348499,0.041663
1,1.254975,0.039470
2,1.240011,0.038869
3,1.230600,0.038614
4,1.118459,0.036322
5,1.119682,0.036517
6,1.211307,0.038948
7,1.216493,0.039422
8,1.233165,0.039708
9,1.247041,0.039644


In [126]:
pls3 = PLS(x_b3, y_b3, names, is_scale)
pls3.pls()
#pls3.report_coe()
pls3.test_err(x1_3, y1_3)

,err_mean,err_std
0,1.170124,0.037194
1,1.274410,0.040146
2,1.364737,0.042463
3,1.486168,0.046163
4,1.508629,0.046939
5,1.521345,0.047595
6,1.531912,0.048130
7,1.525619,0.047987
8,1.531885,0.048128
9,1.531396,0.048059
